<div class="alert alert-block alert-success">
    <p><b>IRONHACK - DATA ANALYSIS FT - JAN22</b></p>
    <i><b>PROJECT:</b> Data cleaning and manipulation - Shark Attack</i>
</div>

---

Our goal with this data cleaning and manipulation project a few questions from a business perspective, assuming our company has not been established yet.

Therefore, at first, we need to identify on what two or three countries we have the highest shark attack occurrences and, moreover, if they were accidental (unprovoked) or provoked.

As stated above, first of all, we need to identify where would our potential clients be located at, what should lead to the following answers:

    1.  What countries have the most shark attacks?
    2.  What countries and cities have the deadliest shark attacks?

Once we have the answers for that we should be ready to answer the next logical question:

    3.Where should our company install its headquarters at?

After deciding where we should install our headquarters at we'll need to decide on what type of business activity we should aim at. In order for that to be possible, we'll need to answer the following questions:

    4.  What would be our target audience? (Government, customers (male, female, does not matter?), ecc) It depends on what triggered the attack: Predatory or provocative behavior?

    5.  What are shark attacks progression through years?
    6.  What season of the year are shark attacks more prevalent?
    7. Is there any gender relationship between shark attacks? How likely are men and women to be attacked?

Once we have answered the questions above we should be ready to decide:

    8.  What would be our company's business purpose? Should we sell a product (i.e. Shark repelent), provide a service to warn or educate bathers/swimmers

Considering that we'd like to enrich this project, as for our future plans we'd like to deepen our research and check how does shark attacks relate to moon phases? This currently is our work in progress.

<div class="alert alert-block alert-info">
    <p><b>Import Libs:</b></p>
</div>

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib as plt
import re
import time
from pandas_profiling import ProfileReport
from datetime import datetime
from bs4 import BeautifulSoup
import dateparser
import requests

<div class="alert alert-block alert-info">
<b>Read CSV and identify encodig:</b></div>

In [3]:
shark_data = pd.read_csv("data/attacks.csv", encoding="latin1")
#DataSet documentation: https://www.sharkattackfile.net/
#mp_cols = "New Moon","First Quarter","Full Moon", "Third Quarter"
#
#Work in progress

#for year in tqdm(range(1800,2020)):
    #globals()[f"df{year}"] = pd.read_html(f'https://www.timeanddate.com/moon/phases/?year={year}')[1]

# for year in range(1800,2020):
#     for mp in mp_cols:
#         globals()f'df{year}{[mp]}' = (globals()[f'df{year}{[mp]}']+f'/{year}').str.replace(' ','/')
#         globals()f'df{year}{[mp]}'.apply(lambda y: datetime.strptime(y, '%d/%b/%Y') if isinstance(y,str) and not y.startswith('*/All/times/') else np.nan)


#    pd.to_datetime(moonp[mp])


#df['New Moon'].apply(lambda x: datetime.strptime(x, '%d/%b/%Y') if isinstance(x,str) and not x.startswith('*/All/times/') else np.nan)


<div class="alert alert-block alert-success">
<b>DataSet Preview:</b> Head, info, describe and dtypes</div>

In [32]:
shark_data.head(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [33]:
shark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [34]:
shark_data.describe()

,Year,original order
count,6300.000000,6309.000000
mean,1927.272381,3155.999683
std,281.116308,1821.396206
min,0.000000,2.000000
25%,1942.000000,1579.000000
50%,1977.000000,3156.000000
75%,2005.000000,4733.000000
max,2018.000000,6310.000000


In [37]:
shark_data.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [16]:
sns.heatmap(shark_data.isnull())

<AxesSubplot:>

<div class="alert alert-block alert-success">
<b>Data cleaning and manipulation.</b></div>

<div class="alert alert-block alert-warning">
    <p> Identify and drop null or disposable rows and columns.</p>
</div>

In [4]:
sns.heatmap(shark_data.isnull())
#Show nulls as an image

<AxesSubplot:>

<div class="alert alert-block alert-warning">
    <p> Identify and modify data types, if needed.</p>
</div>

In [35]:
#check data types
shark_data.info()
#shark_data.astype({"Date":"datetime6", "Year":"int64","Age":"int64"})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

<div class="alert alert-block alert-warning">
    <p>Identify droppable columns / blank cells.</p></div>

In [1]:
null_cols = shark_data.isnull().sum(axis=1) #crio a máscara que soma a quantidade de dados nulos
null_cols[null_cols > 0] #aplico a máscara
shark_data.drop()

NameError: name 'shark_data' is not defined